# 推荐算法

In [168]:
import numpy as np
data = np.array([[5, 4, 0, 3, 0],
       [4, 2, 4, 3, 1],
       [5, 0, 5, 5, 3],
       [4, 1, 0, 0, 1]])
data
# 行为用户 列为 评分

array([[5, 4, 0, 3, 0],
       [4, 2, 4, 3, 1],
       [5, 0, 5, 5, 3],
       [4, 1, 0, 0, 1]])

# 1基于用户的算法

In [169]:
#  。计算相似矩阵
corr = np.corrcoef(data)
corr

array([[ 1.        ,  0.28317606, -0.15861032,  0.63444127],
       [ 0.28317606,  1.        ,  0.66513304,  0.25671802],
       [-0.15861032,  0.66513304,  1.        ,  0.09722222],
       [ 0.63444127,  0.25671802,  0.09722222,  1.        ]])

In [171]:
def user_base_recomd(data, user, corr, topk=2):
    m, n = data.shape
    not_points = [i for i, value in enumerate(data[user]) if value==0]
    if not not_points:
        raise ValueError("The User don't need any recomd...")
    
    result = {}
    for not_point in not_points:
        score = 0.0
        for index, value in enumerate(data[:, not_point]):
            if value !=0 :
                score += value * corr[index, user]
        result[str(not_point)] = score
    results = sorted(result.items(), key=lambda row: row[1], reverse=True)
    return results[:topk]
    
user_base_recomd(data, user=3, corr=corr, topk=2)

[('3', 3.159588963050062), ('2', 1.5129831727316518)]

# 2基于物品的推荐算法

In [172]:
item_corr = np.corrcoef(data.T)
# 物品间的相似度系数
from logging import log
import logging
# logging.info("hehe")

In [173]:
def item_base_recomd(data, user, corr=item_corr, topk=2):
    m, n =data.shape
    if user > m-1:
        raise ValueError("The value of user must < {}, but {} > {}".format(m, user, m-1))

    not_points = [index for index, value in enumerate(data[user]) if value==0]
    if not not_points:
        raise KeyError("E, not need ...")
    logging.info("not points {}".format(not_points))
    
    result = {}
    for not_point in not_points:
        score = 0.0
        for index, point in enumerate(data[user]):
            if point != 0:
                score += point * corr[not_point, index]
        result[not_point] = score
        # not_point 可以不用str
    result = sorted(result.items(), key=lambda row: row[1], reverse=True)
    
    if len(result) <=topk:
        return result
    else:
        return result[:topk]

In [174]:
print("item_base recom", item_base_recomd(data, user=1, topk=1))
print("user_bace recom", user_base_recomd(data, user=1, corr=corr, topk=1))

# 两者推荐的结论不一致。

KeyError: 'E, not need ...'

# 3基于余弦相似度的推荐计算

In [175]:
# test 
data = np.array([[4, 3, 0, 5, 0], [5, 0, 4, 4, 0], [4, 0, 5, 0, 3],
                [2, 3, 0, 1, 0], [0, 4, 2, 0, 5]])

In [176]:
def calu_cossim(vec1, vec2):
    v12_doc = np.sum(np.dot(vec1, vec2))
    vec1_norm = np.sqrt(np.sum(np.power(vec1, 2)))
    vec2_norm = np.sqrt(np.sum(np.power(vec2, 2)))
    return  v12_doc / (vec1_norm * vec2_norm + 0.01)

In [182]:
def calu_cossim_matrix(data, func=calu_cossim):
    m, n = data.shape
    cos_matirx = np.zeros((m, m))
    for i in range(0, m):
        for j in range(i+1, m):
            cos_matirx[j, i] = calu_cossim(data[i], data[j])
    return np.round(cos_matirx, 2)
cos_matrix = calu_cossim_matrix(data, func=calu_cossim)

In [183]:
item_cossim_matrix = calu_cossim_matrix(data.T, calu_cossim)

In [184]:
print("item_base recom", item_base_recomd(data, user=0, corr=item_cossim_matrix, topk=2))
print("user_bace recom", user_base_recomd(data, user=0, corr=cos_matrix, topk=2))
# 这里使用不同的相似性矩阵，不然会出错的, 利用下三三角存储 也会减少计算量

item_base recom [(2, 3.64), (4, 2.81)]
user_bace recom [('2', 5.1), ('4', 2.21)]


In [185]:
cos_matrix

array([[0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.75, 0.  , 0.  , 0.  , 0.  ],
       [0.32, 0.75, 0.  , 0.  , 0.  ],
       [0.83, 0.5 , 0.3 , 0.  , 0.  ],
       [0.25, 0.16, 0.53, 0.48, 0.  ]])